In [1]:
from langfuse.callback import CallbackHandler
from dotenv import load_dotenv
import os
load_dotenv()

public_key = os.getenv("ENV_PUBLIC_KEY", None)
secret_key = os.getenv("ENV_SECRET_KEY", None)
env_host = os.getenv("ENV_HOST", None)

print(public_key, secret_key, env_host)

pk-lf-4e7c6dde-01b2-4cf3-81b6-fd37741cfb9c sk-lf-71c07a5a-a37a-4228-9788-1d7d06599dda11 None


In [4]:
from langfuse import Langfuse
 
# langfuse = Langfuse(public_key, secret_key, env_host, debug=True)
langfuse = Langfuse(public_key, secret_key, env_host)

In [ ]:
from datetime import datetime
from langfuse.model import InitialGeneration, Usage
 
generationStartTime = datetime.now()
 
# call to an LLM API
 
langfuse.generation(InitialGeneration(
    name="summary-generation",
    startTime=generationStartTime,
    endTime=datetime.now(),
    model="gpt-3.5-turbo",
    modelParameters={"maxTokens": "1000", "temperature": "0.9"},
    prompt=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals..."}],
    completion="The Q3 OKRs contain goals for multiple teams...",
    usage=Usage(promptTokens=50, completionTokens = 49),
    metadata={"interface": "whatsapp"}
))

ERROR:langfuse:1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Traceback (most recent call last):
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/client.py", line 276, in create_trace
    return self.client.trace.create(request=request)
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/api/resources/trace/client.py", line 53, in create
    raise UnauthorizedError(pydantic.parse_obj_as(str, _response.json()))  # type: ignore
  File "pydantic/tools.py", line 38, in pydantic.tools.parse_obj_as
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
ERROR:langfuse:1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Traceback (most recent call last):
  File "/Users/guangyaliu/go/src/github.com/langfuse/l

-----------------------------------------------
id='760f151f-84b4-4364-8347-29587fcde8f7' trace_id='eb10b20c-2ebd-4b53-97a6-130def8ee003' trace_id_type=None name='summary-generation' start_time=datetime.datetime(2023, 11, 2, 10, 54, 25, 164304) metadata={'interface': 'whatsapp'} input=None output=None level=None status_message=None parent_observation_id=None version=None end_time=datetime.datetime(2023, 11, 2, 10, 54, 25, 164393) completion_start_time=None model='gpt-3.5-turbo' model_parameters={'maxTokens': '1000', 'temperature': '0.9'} prompt=[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals...'}] completion='The Q3 OKRs contain goals for multiple teams...' usage=LlmUsage(prompt_tokens=50, completion_tokens=49, total_tokens=None)
----------------------------------------

ERROR:langfuse:1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Traceback (most recent call last):
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/client.py", line 293, in create_generation
    return self.client.generations.log(request=request)
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/api/resources/generations/client.py", line 52, in log
    raise UnauthorizedError(pydantic.parse_obj_as(str, _response.json()))  # type: ignore
  File "pydantic/tools.py", line 38, in pydantic.tools.parse_obj_as
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)


-----------------------------------------------
{'success': False, 'message': 'Invalid credentials'}
-----------------------------------------------
-----------------------------------------------
id='760f151f-84b4-4364-8347-29587fcde8f7' trace_id='eb10b20c-2ebd-4b53-97a6-130def8ee003' trace_id_type=None name='summary-generation' start_time=datetime.datetime(2023, 11, 2, 10, 54, 25, 164304) metadata={'interface': 'whatsapp'} input=None output=None level=None status_message=None parent_observation_id=None version=None end_time=datetime.datetime(2023, 11, 2, 10, 54, 25, 164393) completion_start_time=None model='gpt-3.5-turbo' model_parameters={'maxTokens': '1000', 'temperature': '0.9'} prompt=[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals...'}] completion='The Q3 OKRs co

ERROR:langfuse:1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Traceback (most recent call last):
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/client.py", line 293, in create_generation
    return self.client.generations.log(request=request)
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/api/resources/generations/client.py", line 52, in log
    raise UnauthorizedError(pydantic.parse_obj_as(str, _response.json()))  # type: ignore
  File "pydantic/tools.py", line 38, in pydantic.tools.parse_obj_as
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)


-----------------------------------------------
{'success': False, 'message': 'Invalid credentials'}
-----------------------------------------------
-----------------------------------------------
id='760f151f-84b4-4364-8347-29587fcde8f7' trace_id='eb10b20c-2ebd-4b53-97a6-130def8ee003' trace_id_type=None name='summary-generation' start_time=datetime.datetime(2023, 11, 2, 10, 54, 25, 164304) metadata={'interface': 'whatsapp'} input=None output=None level=None status_message=None parent_observation_id=None version=None end_time=datetime.datetime(2023, 11, 2, 10, 54, 25, 164393) completion_start_time=None model='gpt-3.5-turbo' model_parameters={'maxTokens': '1000', 'temperature': '0.9'} prompt=[{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals...'}] completion='The Q3 OKRs co

ERROR:langfuse:1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Traceback (most recent call last):
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/client.py", line 293, in create_generation
    return self.client.generations.log(request=request)
  File "/Users/guangyaliu/go/src/github.com/langfuse/langfuse-python/langfuse/api/resources/generations/client.py", line 52, in log
    raise UnauthorizedError(pydantic.parse_obj_as(str, _response.json()))  # type: ignore
  File "pydantic/tools.py", line 38, in pydantic.tools.parse_obj_as
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.str)
Giving up _execute_task_with_backoff(...) after 3 tries (pydantic.error_wrappers.ValidationError: 1 validation error for ParsingModel[str]
__root__
  str type expected (type=type_error.

-----------------------------------------------
{'success': False, 'message': 'Invalid credentials'}
-----------------------------------------------


In [4]:
from langfuse.model import CreateTrace, CreateSpan, CreateGeneration, CreateEvent
 
trace = langfuse.trace(CreateTrace(name = "llm-feature"))
retrieval = trace.span(CreateSpan(name = "retrieval"))
retrieval.generation(CreateGeneration(name = "query-creation"))
retrieval.span(CreateSpan(name = "vector-db-search"))
retrieval.event(CreateEvent(name = "db-summary"))
trace.generation(CreateGeneration(name = "user-output"))

In [41]:
from langfuse.model import CreateTrace
 
trace = langfuse.trace(CreateTrace(
    name = "docs-retrieval",
    userId = "user__935d7d1d-8625-4ef4-8651-544613e7bd22",
    metadata = {
        "env": "production",
        "email": "user@langfuse.com",
    }
))

In [26]:
from datetime import datetime
from langfuse.model import CreateSpan, UpdateSpan
 
retrievalStartTime = datetime.now()
 
# retrieveDocs = retrieveDoc()
# ...
 
span = trace.span(CreateSpan(
        name="embedding-search",
        startTime=retrievalStartTime,
        endTime=datetime.now(),
        metadata={"database": "pinecone"},
        input = {'query': 'This document entails the OKR goals for ACME'},
    )
)

In [27]:
span = span.update(UpdateSpan(
        output = {"response": "[{'name': 'OKR Engineering', 'content': 'The engineering department defined the following OKR goals...'},{'name': 'OKR Marketing', 'content': 'The marketing department defined the following OKR goals...'}]"}
    )
)

In [9]:
from langfuse.model import CreateGeneration, Usage, UpdateGeneration
 
from datetime import datetime
 
generationStartTime = datetime.now()
 
# chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "Hello world"}])
# ...
 
generation = trace.generation(CreateGeneration(
    name="summary-generation",
    startTime=generationStartTime,
    endTime=datetime.now(),
    model="gpt-3.5-turbo",
    modelParameters={"maxTokens": "1000", "temperature": "0.9"},
    prompt=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Please generate a summary of the following documents \nThe engineering department defined the following OKR goals...\nThe marketing department defined the following OKR goals..."}],
    metadata={"interface": "whatsapp"}
))

In [10]:
generation.update(UpdateGeneration(
    completion="The Q3 OKRs contain goals for multiple teams...",
    usage=Usage(promptTokens=50, completionTokens = 49),
))

In [11]:
from langfuse.model import CreateEvent
from datetime import datetime
 
event = span.event(CreateEvent(
        name="chat-docs-retrieval",
        startTime=datetime.now(),
        metadata={"key": "value"},
        input = {"key": "value"},
        output = {"key": "value"}
    )
)

In [12]:
from langfuse.model import CreateScore
 
trace.score(CreateScore(
    name="user-explicit-feedback",
    value=1,
    comment="I like how personalized the response is"
))

In [15]:
!pip install fastapi

In [16]:
from contextlib import asynccontextmanager
from fastapi import FastAPI, Query, BackgroundTasks
from langfuse.model import InitialGeneration
 
 
@asynccontextmanager
async def lifespan(app: FastAPI):
    # Operation on startup
 
    yield  # wait until shutdown
 
    # Flush all events to be sent to Langfuse on shutdown and terminate all Threads gracefully. This operation is blocking.
    langfuse.flush()
 
 
app = FastAPI(lifespan=lifespan)

In [17]:
langfuse = Langfuse(public_key, secret_key, env_host, debug=True)
 
@app.get("/generate/",tags=["APIs"])
async def campaign(prompt: str = Query(..., max_length=20)):
  # call to a LLM
  generation = langfuse.generation(
      InitialGeneration(name="llm-feature", metadata="test", prompt=prompt)
  )
  return True